# Tech Challenge 3: Arquitetura ML e Aprendizado

## Introdução

*Problema a ser resolvido*

Identificar a correlação entre dados de usuário (idade, gênero, ocupação), preferência por filmes e sua avaliação.

A partir desse modelo, será criada uma API que poderá predizer determinado resultado baseado nos dados pertencentes de uma pessoa (idade, gênero, ocupação) e sugerir filmes que se encaixam com as preferencias do mesmo.

## Setup

In [ ]:
%pip install matplotlib
%pip install seaborn
%pip install scikit-learn
%pip install xgboost
%pip install requests
%pip install boto3
%pip install python-dotenv

In [356]:
import pandas as pd
import zipfile
import numpy as np
import requests
import os
import boto3
from io import StringIO
import pickle
from dotenv import load_dotenv
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Perceptron
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [ ]:
load_dotenv(override=True)

## Load dos dados

*Conjunto de Dados*

Usaremos o grupo de dados do Movielens para esse caso de estudo (https://files.grouplens.org/datasets/movielens/)

In [358]:
url = "https://files.grouplens.org/datasets/movielens/ml-1m.zip"
file_path = "ml-1m.zip"
response = requests.get(url)
with open(file_path, 'wb') as file:
        file.write(response.content)

In [359]:
with zipfile.ZipFile('ml-1m.zip', 'r') as zip_ref:
    zip_ref.extractall('raw')

In [360]:
os.remove(file_path)

In [ ]:
user_data = pd.read_csv('raw/ml-1m/users.dat',sep='::',header=None,names=['userid','gender','age','occupation','zipcode'],encoding='iso-8859-2')
movie_data = pd.read_csv('raw/ml-1m/movies.dat',sep='::',header=None,names=['movieid','title','genres'],encoding='iso-8859-2')
ratings_data = pd.read_csv('raw/ml-1m/ratings.dat',sep='::',header=None,usecols=[0,1,2],names=['userid','movieid','rating'],encoding='iso-8859-2')


## Pre-processamento

### Estrutura de cada dataset

In [ ]:
user_data.head()

In [ ]:
movie_data.head()

In [ ]:
ratings_data.head()

### Analise dos dados

Ao analisar os dados, podemos perceber que possuímos um ratio desbalanceado entre usuários homens (M) e usuárias mulheres (F).

In [ ]:
user_data['gender'].value_counts().plot(kind='pie', autopct='%1.0f%%')

Antes de prosseguirmos uma outra informação que é interessante de analisarmos é a distribuição etária de nossos usuários. Vale a pena resaltar que a coluda idade (age) já está "normalizada", obedecendo a seguinte proporção:
*  1:  Mais novos que 18 anos
* 18:  18-24 anos
* 25:  25-34 anos
* 35:  35-44 anos
* 45:  45-49 anos
* 50:  50-55 anos
* 56:  56+ anos

In [ ]:
user_data.plot.hist(column=["age"], by="gender", figsize=(10, 8))

### Normaliza os dados

Ao analisarmos os dados, mais precisamente a coluna *genres* do dataset de Filmes, é possível perceber que temos um problema de 'Multi-Label' nos nossos dados, onde um filme pode pertencer a 1 ou N gêneros de filmes. Para contornarmos essa situação, iremos converter essa coluna para uma lista de string:

In [368]:
movie_data['genres'] = movie_data["genres"].str.split("|")

In [ ]:
movie_data.head()

Em seguida, utilizaremos o **MultiLabelBinarizer** para transformar essa coluna em uma matriz binária

In [370]:
mlb = MultiLabelBinarizer()
encoded_data = mlb.fit_transform(movie_data['genres'])
dados_encodados = pd.DataFrame(encoded_data, columns=mlb.classes_)

In [ ]:
dados_encodados.head()

E finalmente concatenaremos esse novo dataframe ao "original", podendo assim excluir a coluna '*genres*' de nossos dados.

In [ ]:
movie_data = pd.concat([movie_data,dados_encodados], axis=1)
movie_data = movie_data.drop(['genres'], axis=1)
movie_data.head()

Partindo de um princípio parecido, a coluna Gender do dataset de usuário que está dividido em M e F, pode ser alterada um LabelEncoder para facilitar nosso treinamento de modelos.

In [373]:
label_encoder = LabelEncoder()
user_data['gender'] = label_encoder.fit_transform(user_data['gender'])

In [ ]:
user_data['gender'].head()

### Merge e limpeza dos dados

Criaremos um dataset/sample contendo tanto dados de usuarios quanto as suas respectivas avaliacoes. para treinarmos e testarmos os nossos modelos posteriomente.

In [ ]:
dados_completos = pd.merge(user_data, ratings_data, on='userid')
dados_completos = pd.merge(dados_completos, movie_data, on='movieid')
dados_completos = dados_completos.sample(n=5000, random_state=23)
dados_completos

In [ ]:
dados_modelo = dados_completos.drop(['movieid', 'userid', 'title', 'zipcode'], axis=1)
dados_modelo

In [ ]:
dados_modelo.isna().sum()

### Separa dados de treino de dados de teste

Agora, podemos começar a nossa jornada de validação e testes de alguma modelos, como temos a coluna rating, que funciona como a nota do especialista, podemos começar com um aprendizado supervisionado. 

Começaremos com modelos de regressão para tal.

Primeiramente separamos os nossos dados X (features) e Y (target).

In [378]:
x = dados_modelo.drop(columns=['rating'], axis=1)
y = dados_modelo['rating']

Agora separamos a parcela de treino (80%) e teste (20%)

In [379]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=23)

## Utils

Já deixaremos aqui, uma função para a avaliação do Modelo e uma função para busca dos melhores hiper parâmetros

In [362]:
def avalia_modelo(y_test,y_pred, tipo_modelo):
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    print(f'{tipo_modelo} - MSE: {mse} RMSE: {rmse} MAPE: {mape}')

def procura_melhores_parametros(model, parametros, treino_x, treino_y):
    grid_search = GridSearchCV(estimator=model,
                           param_grid=parametros,
                           cv=5,
                           scoring='neg_mean_absolute_percentage_error')

    # busca a melhor combinação de parametros, baseado nos dados de teste
    grid_search.fit(treino_x, treino_y)

    # retorna o melhor modelo
    melhor_modelo = grid_search.best_estimator_
    return melhor_modelo  

def save_dataframe_as_dat_locally(dataframe, file_path):
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    
    with open(file_path, 'w', encoding='iso-8859-2') as f:
         for index, row in dataframe.iterrows():
            f.write('::'.join(map(str, row.values)) + '\n')
    print(f"Arquivo {file_path} salvo com sucesso.")

def create_boto_client(bucket_name):
    boto_client = boto3.client(
        's3',
        aws_access_key_id=os.getenv('s3_access_key_id'),
        aws_secret_access_key=os.getenv('s3_secret_access_key'),
        aws_session_token=os.getenv('s3_session_token')
    )

    try:
        boto_client.list_objects_v2(Bucket=bucket_name)
        return boto_client
    except Exception as e:
        return None

## Algoritmos

***REGRESSÂO LINEAR***

Começaremos os nossos testes, utilizando o LinearRegression do scikit-learn

In [380]:
model_lr = procura_melhores_parametros(LinearRegression(), {}, x_train, y_train)
lr_pred =  model_lr.predict(x_test)

***KNN Regressor***

Agora utilizaremos o KNN Regressor, contudo para esse modelo utilizaremos a função previamente criada para chegar no melhor número de "vizinhos"

In [381]:

parametros = {
    'n_neighbors': [2, 5, 8, 10],
    'metric':['euclidean']
}

model_knn = procura_melhores_parametros(KNeighborsRegressor(), parametros, x_train, y_train)
knnregressor_pred = model_knn.predict(x_test)

***Support Vector Machine: Linear***

O SVM é uma ferramenta robusta, poderosa e versátil. Indicada para problemas mais complexos e alcançar alta precisão, vamos ver como se sai em nosso caso de estudo. Mais uma vez obtendo o melhor modelo a partir do gridsearch

In [382]:
parametros = {
    'C': [5, 10]
}

model_svm = procura_melhores_parametros(SVR(kernel='linear'), parametros, x_train, y_train)
svr_pred =  model_svm.predict(x_test)

***Árvore de Decisão***

A árvore de decisão contrói uma estrutura hierárquica de regras que dividem em subconjuntos baseados nas características, facilitando a identificação.

In [383]:
parametros = {}

model_tree = procura_melhores_parametros(DecisionTreeClassifier(), parametros, x_train, y_train)
decision_tree_pred =  model_tree.predict(x_test)

***Redes Neurais Artificiais (RNA)***

As redes neurais artificiais são modelos computacionais que, atraves dos Perceptrons, imitam o funcionamento do cérebro humano para resolver problemas complexos. 

In [384]:
parametros = {}

model_perceptron = procura_melhores_parametros(Perceptron(), parametros, x_train, y_train)
perceptron_pred =  model_perceptron.predict(x_test)

***Multilayer Perceptron (MLP)***

O multilayer percepton é um modelo de rede neural artificial com múltiplas camadas

In [ ]:
parametros = {}

model_mlp = procura_melhores_parametros(MLPClassifier(), parametros, x_train, y_train)
mlp_pred =  model_mlp.predict(x_test)

***XGBoost***

Finalizaremos nossos testes com diferente tipos de estimadores, fazendo uso do XGBoost. É um dos mais utilizados pelo mercado e academia, funciona atráves da construção sequencial de N árvores de decisão mais fracas, onde cada árvore aprende com o erro da anterior.

In [386]:
parametros = {
    "n_estimators": [20, 50, 100],
    "max_depth": [3, 6, 8],
    "learning_rate": [0.1]
}

model_xgb = procura_melhores_parametros(XGBRegressor(), parametros, x_train, y_train)

xgboost_pred = model_xgb.predict(x_test)

## Resultado

Agora, uma vez que temos os estimadores treinados, mas definir através do MAPE. Qual modelo performa melhor em nosso cenário de estudo:

In [ ]:
avalia_modelo(y_test, lr_pred, "LinearRegression")
avalia_modelo(y_test, knnregressor_pred, "KNN Regressor")
avalia_modelo(y_test, svr_pred, "SVM Linear")
avalia_modelo(y_test, xgboost_pred, "XGBoost")
avalia_modelo(y_test, decision_tree_pred, "DecisionTree")
avalia_modelo(y_test, perceptron_pred, "Perceptron")
avalia_modelo(y_test, mlp_pred, "MLP")

Podemos finalmente exportar nosso modelo e avançar para a próxima etapa que é construir uma API que consiga predizer para um usuário de acordo com diferentes dados de entrada.

## Upload dataset e modelo para o S3

In [ ]:
bucket_name = 'techchallengegp53'

boto_client = create_boto_client(bucket_name)
aws_connected = boto_client is not None

print(f'AWS Connected {aws_connected}')

Salvando sample no S3

In [389]:
if aws_connected:

    user_data_buffer = StringIO()

    for index, row in user_data.iterrows():
        user_data_buffer.write('::'.join(map(str, row.values)) + '\n')
    boto_client.put_object(Bucket=bucket_name, Key="refined/user_data.dat", Body=user_data_buffer.getvalue())

    movie_data_buffer = StringIO()

    for index, row in movie_data.iterrows():
        movie_data_buffer.write('::'.join(map(str, row.values)) + '\n')
    boto_client.put_object(Bucket=bucket_name, Key="refined/movie_data.dat", Body=movie_data_buffer.getvalue())

Salva modelos na S3

In [390]:
if aws_connected:
    modelos_salvar = {
        'LinearRegression': model_lr,
        'KNN Regressor': model_knn,
        'SVM Linear': model_svm,
        'XGBoost': model_xgb,
        'DecisionTree': model_tree,
        'Perceptron': model_perceptron,
        'MLP': None
    }

    for nome_modelo, instancia_modelo in modelos_salvar.items():
        modelo_buffer = pickle.dumps(instancia_modelo)
        boto_client.put_object(Bucket=bucket_name, Key=f"modelo/{nome_modelo}.sav", Body=modelo_buffer)

Verificar se existe conexão com o Bucket. 
Se não existir, savamos local no `refined`

In [353]:
if not aws_connected:
    bucket_folder = 'refined'

    user_data_file_path = os.path.join(bucket_folder, 'user_data.dat')
    movie_data_file_path = os.path.join(bucket_folder, 'movie_data.dat')

    save_dataframe_as_dat_locally(user_data, user_data_file_path)
    save_dataframe_as_dat_locally(movie_data, movie_data_file_path)

## Cleanup

In [354]:
import shutil

pastas_remocao = ['raw', 'refined']

for pasta in pastas_remocao:
    if (os.path.exists(pasta)):
        shutil.rmtree(pasta)